
## 1. Import Libraries


In [2]:

import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')


## 2. Data Collection

In [1]:

tickers = {
    'S&P 500': '^GSPC',
    'FTSE 100': '^FTSE',
    'Nikkei 225': '^N225',
    'Gold ETF': 'GLD',
    'US Treasury Bonds': 'TLT'
}
start_date = '2010-01-01'
end_date = '2020-12-31'


In [14]:
# Download data for each ticker
data = {}
for asset, ticker in tickers.items():
    data[asset] = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [15]:
# Function to calculate log returns
def calculate_log_returns(df):
    df['Log Return'] = df['Close'].pct_change().apply(lambda x: np.log(1 + x))
    return df

# Calculate log returns for each asset
for asset in data:
    data[asset] = calculate_log_returns(data[asset])

print(data['S&P 500'].tail())

Price             Close         High          Low         Open      Volume  \
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC   
Date                                                                         
2020-12-23  3690.010010  3711.239990  3689.280029  3693.419922  3779160000   
2020-12-24  3703.060059  3703.820068  3689.320068  3694.030029  1883780000   
2020-12-28  3735.360107  3740.510010  3723.030029  3723.030029  3535460000   
2020-12-29  3727.040039  3756.120117  3723.310059  3750.010010  3393290000   
2020-12-30  3732.040039  3744.629883  3730.209961  3736.189941  3154850000   

Price      Log Return  
Ticker                 
Date                   
2020-12-23   0.000746  
2020-12-24   0.003530  
2020-12-28   0.008685  
2020-12-29  -0.002230  
2020-12-30   0.001341  


In [16]:
# Add 5-day and 21-day moving averages for each asset
for asset, df in data.items():
    df['5-day MA'] = df['Close'].rolling(window=5).mean()
    df['21-day MA'] = df['Close'].rolling(window=21).mean()


print(data['S&P 500'].tail())


Price             Close         High          Low         Open      Volume  \
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC   
Date                                                                         
2020-12-23  3690.010010  3711.239990  3689.280029  3693.419922  3779160000   
2020-12-24  3703.060059  3703.820068  3689.320068  3694.030029  1883780000   
2020-12-28  3735.360107  3740.510010  3723.030029  3723.030029  3535460000   
2020-12-29  3727.040039  3756.120117  3723.310059  3750.010010  3393290000   
2020-12-30  3732.040039  3744.629883  3730.209961  3736.189941  3154850000   

Price      Log Return     5-day MA    21-day MA  
Ticker                                           
Date                                             
2020-12-23   0.000746  3700.815967  3674.680466  
2020-12-24   0.003530  3696.931982  3677.901902  
2020-12-28   0.008685  3702.122021  3682.935721  
2020-12-29  -0.002230  3708.546045  3687.159052  
2020-12-30   0.001341  37

In [17]:
# Add 21-day rolling volatility for each asset
for asset, df in data.items():
    df['Volatility'] = df['Log Return'].rolling(window=21).std()


print(data['S&P 500'].tail())


Price             Close         High          Low         Open      Volume  \
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC   
Date                                                                         
2020-12-23  3690.010010  3711.239990  3689.280029  3693.419922  3779160000   
2020-12-24  3703.060059  3703.820068  3689.320068  3694.030029  1883780000   
2020-12-28  3735.360107  3740.510010  3723.030029  3723.030029  3535460000   
2020-12-29  3727.040039  3756.120117  3723.310059  3750.010010  3393290000   
2020-12-30  3732.040039  3744.629883  3730.209961  3736.189941  3154850000   

Price      Log Return     5-day MA    21-day MA Volatility  
Ticker                                                      
Date                                                        
2020-12-23   0.000746  3700.815967  3674.680466   0.006239  
2020-12-24   0.003530  3696.931982  3677.901902   0.005307  
2020-12-28   0.008685  3702.122021  3682.935721   0.005537  
2020-12-2

In [18]:
# Drop rows with missing values 
for asset in data:
    data[asset].dropna(inplace=True)


print(data['S&P 500'].tail())


Price             Close         High          Low         Open      Volume  \
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC   
Date                                                                         
2020-12-23  3690.010010  3711.239990  3689.280029  3693.419922  3779160000   
2020-12-24  3703.060059  3703.820068  3689.320068  3694.030029  1883780000   
2020-12-28  3735.360107  3740.510010  3723.030029  3723.030029  3535460000   
2020-12-29  3727.040039  3756.120117  3723.310059  3750.010010  3393290000   
2020-12-30  3732.040039  3744.629883  3730.209961  3736.189941  3154850000   

Price      Log Return     5-day MA    21-day MA Volatility  
Ticker                                                      
Date                                                        
2020-12-23   0.000746  3700.815967  3674.680466   0.006239  
2020-12-24   0.003530  3696.931982  3677.901902   0.005307  
2020-12-28   0.008685  3702.122021  3682.935721   0.005537  
2020-12-2

## 3. Merge Features

In [19]:
# Combine data for all assets into a single DataFrame
merged_data = pd.DataFrame()

for asset, df in data.items():
    asset_data = df[['Log Return', '5-day MA', '21-day MA', 'Volatility']]
    asset_data.columns = [f'{asset} Log Return', f'{asset} 5-day MA', f'{asset} 21-day MA', f'{asset} Volatility']
    merged_data = pd.concat([merged_data, asset_data], axis=1)


print(merged_data.tail())


            S&P 500 Log Return  S&P 500 5-day MA  S&P 500 21-day MA  \
Date                                                                  
2020-12-24            0.003530       3696.931982        3677.901902   
2020-12-25                 NaN               NaN                NaN   
2020-12-28            0.008685       3702.122021        3682.935721   
2020-12-29           -0.002230       3708.546045        3687.159052   
2020-12-30            0.001341       3717.502051        3692.416678   

            S&P 500 Volatility  FTSE 100 Log Return  FTSE 100 5-day MA  \
Date                                                                     
2020-12-24            0.005307             0.000969        6498.900000   
2020-12-25                 NaN                  NaN                NaN   
2020-12-28            0.005537                  NaN                NaN   
2020-12-29            0.005586             0.015353        6509.220020   
2020-12-30            0.005428            -0.007129       

In [20]:
for asset in data:
    merged_data.dropna(inplace=True)

In [21]:
merged_data.head()

,S&P 500 Log Return,S&P 500 5-day MA,S&P 500 21-day MA,S&P 500 Volatility,FTSE 100 Log Return,FTSE 100 5-day MA,FTSE 100 21-day MA,FTSE 100 Volatility,Nikkei 225 Log Return,Nikkei 225 5-day MA,Nikkei 225 21-day MA,Nikkei 225 Volatility,Gold ETF Log Return,Gold ETF 5-day MA,Gold ETF 21-day MA,Gold ETF Volatility,US Treasury Bonds Log Return,US Treasury Bonds 5-day MA,US Treasury Bonds 21-day MA,US Treasury Bonds Volatility
Date,,,,,,,,,,,,,,,,,,,,
2010-02-03,-0.005489,1089.637988,1119.278570,0.010256,-0.005713,5223.620020,5380.666690,0.008208,0.003200,10318.553906,10614.118583,0.012431,-0.003948,107.723999,109.350476,0.011520,-0.011663,59.371735,58.909332,0.008266
2010-02-04,-0.031636,1085.353979,1115.782854,0.012106,-0.021921,5222.339941,5362.061919,0.009191,-0.004658,10306.891992,10598.601935,0.012424,-0.040649,107.301999,109.096667,0.014478,0.015703,59.448825,58.962280,0.008830
2010-02-05,0.002893,1083.817969,1112.404279,0.012153,-0.015373,5196.819922,5339.880952,0.009518,-0.029286,10278.701953,10566.489537,0.013726,0.002966,107.045999,108.771428,0.013895,0.002177,59.448164,59.058796,0.008194
2010-02-08,-0.008903,1077.327979,1108.359044,0.012104,0.006185,5165.799902,5318.838077,0.009715,-0.010522,10228.062109,10531.735259,0.013819,-0.006133,106.184000,108.448571,0.013894,0.001304,59.571834,59.154398,0.008194
2010-02-09,0.012956,1070.767993,1104.813331,0.012568,0.003822,5131.499902,5298.538063,0.009813,-0.001903,10140.424219,10490.524786,0.013437,0.013082,105.440001,108.164762,0.014236,-0.010039,59.541898,59.222720,0.008574


## 4. Feature & Target Preparation

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Drop the last row from both the features and target to align their lengths
features = merged_data.values[:-1]  # Drop the last row of features to match target length
target = merged_data['S&P 500 Log Return'].shift(-1).dropna().values  # Shift target and drop NaN values

# Ensure that both features and target arrays have the same length after dropping the last row
assert len(features) == len(target), f"Features and target length mismatch: {len(features)} != {len(target)}"


In [23]:
# Normalize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Split data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, shuffle=False)

# Reshape the data for LSTM input (samples, timesteps, features)
X_train_lstm = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Check the shape of the data
print(X_train_lstm.shape, X_test_lstm.shape)


(2028, 1, 20) (508, 1, 20)


In [24]:

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_lstm, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_lstm, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


## 5. LSTM Model

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Build the LSTM model
model_lstm = Sequential()

# Add LSTM layer (units = 50, return_sequences=False to output only the final time step)
model_lstm.add(LSTM(units=50, return_sequences=False, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))

# Add Dense layer for prediction
model_lstm.add(Dense(units=1))  # Predicting a single value (next day's return)

# Compile the model
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
history_lstm = model_lstm.fit(X_train_lstm, y_train, epochs=10, batch_size=32, validation_data=(X_test_lstm, y_test))

# Evaluate the model on the test set
lstm_loss = model_lstm.evaluate(X_test_lstm, y_test)
print("LSTM Model Loss:", lstm_loss)

y_pred_lstm = model_lstm.predict(X_test_lstm)


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0057 - val_loss: 0.0039
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.2764e-04 - val_loss: 0.0020
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.5398e-04 - val_loss: 0.0016
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.8195e-04 - val_loss: 0.0015
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.6113e-04 - val_loss: 0.0013
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.4396e-04 - val_loss: 0.0014
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.2758e-04 - val_loss: 0.0012
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.3814e-04 - val_loss: 0.0013
Epoch 9/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.1124e-04 - val_loss: 0.0012
Epoch 10/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.1491e-04 - val_loss: 0.0011
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.4251e-04 
LSTM Model Loss: 0.0010999138467013836
16/16 ━━━━━━━━━━━━━━━━━

## Transformer

In [27]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, output_dim):
        super(TransformerModel, self).__init__()
        
        # Transformer block (No target input, only source)
        self.transformer = nn.Transformer(d_model=input_dim, nhead=num_heads, num_encoder_layers=num_layers)
        
        # Fully connected layer for output
        self.fc = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        # x is of shape (batch_size, seq_len, input_dim)
        x = x.permute(1, 0, 2)  # Change shape to (seq_len, batch_size, input_dim) for Transformer
        
        # Apply the Transformer
        # Since we don't need a target (tgt), we will use the same input `x` as the input to the Transformer
        x = self.transformer(x, x)  # Pass the same tensor as both source and target
        
        # Pooling across the sequence length (mean pooling)
        x = x.mean(dim=0)  # Pooling across the sequence length
        x = self.fc(x)
        return x


# Initialize model
model_transformer = TransformerModel(input_dim=X_train.shape[1], num_heads=4, num_layers=2, output_dim=1)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model_transformer.parameters(), lr=0.001)


In [30]:
# Training the model
epochs = 50
for epoch in range(epochs):
    model_transformer.train()
    
    # Forward pass
    optimizer.zero_grad()
    y_pred = model_transformer(X_train_tensor)
    
    # Compute loss
    loss = criterion(y_pred, y_train_tensor.view(-1, 1))
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

# Evaluate the Transformer model on test data
model_transformer.eval()
with torch.no_grad():
    y_pred_transformer = model_transformer(X_test_tensor)
    transformer_loss = criterion(y_pred_transformer, y_test_tensor.view(-1, 1))

print("Transformer Model Loss:", transformer_loss.item())


Epoch [1/50], Loss: 0.004986262880265713
Epoch [2/50], Loss: 0.004976753145456314
Epoch [3/50], Loss: 0.004759818781167269
Epoch [4/50], Loss: 0.004259653855115175
Epoch [5/50], Loss: 0.0037758592516183853
Epoch [6/50], Loss: 0.003530823392793536
Epoch [7/50], Loss: 0.003348270198330283
Epoch [8/50], Loss: 0.003415740327909589
Epoch [9/50], Loss: 0.003178624203428626
Epoch [10/50], Loss: 0.00316948676481843
Epoch [11/50], Loss: 0.002968250075355172
Epoch [12/50], Loss: 0.002656942931935191
Epoch [13/50], Loss: 0.002847072435542941
Epoch [14/50], Loss: 0.00249610492028296
Epoch [15/50], Loss: 0.0022535184398293495
Epoch [16/50], Loss: 0.0024818822275847197
Epoch [17/50], Loss: 0.0025071275886148214
Epoch [18/50], Loss: 0.0025469474494457245
Epoch [19/50], Loss: 0.002558793406933546
Epoch [20/50], Loss: 0.002368098823353648
Epoch [21/50], Loss: 0.0023074785713106394
Epoch [22/50], Loss: 0.0021353887859731913
Epoch [23/50], Loss: 0.0020737480372190475
Epoch [24/50], Loss: 0.00191167788580

In [31]:

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])


## 6. Hyperparameter Grid Search

In [32]:
param_grid = {
    'hidden_dim': [64, 128],
    'num_layers': [2, 3],
    'dropout': [0.1, 0.2],
    'learning_rate': [0.0003, 0.0005],
    'epochs': [50],
    'batch_size': [32]
}

best_mse = float('inf')
best_model = None
best_params = {}


In [34]:
for params in ParameterGrid(param_grid):
    print(f"\nTraining with: {params}")
    model = LSTMModel(input_dim=X_train_lstm.shape[2],
                      hidden_dim=params['hidden_dim'],
                      num_layers=params['num_layers'],
                      dropout=params['dropout'])
    
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    criterion = nn.MSELoss()
    
    best_loss = float('inf')
    patience, patience_counter = 10, 0

    for epoch in range(params['epochs']):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train_tensor)
        loss = criterion(y_pred, y_train_tensor.view(-1, 1))
        loss.backward()
        optimizer.step()

        if loss.item() < best_loss:
            best_loss = loss.item()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1} with loss {loss.item()}")
            break

        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{params['epochs']}], Loss: {loss.item():.6f}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred_test = model(X_test_tensor).cpu().numpy().flatten()
        mse = mean_squared_error(y_test, y_pred_test)
        r2 = r2_score(y_test, y_pred_test)
        mae = mean_absolute_error(y_test, y_pred_test)
        print(f"Test MSE: {mse:.6f}, R²: {r2:.4f}, MAE: {mae:.6f}")

        if mse < best_mse:
            best_mse = mse
            best_params = params
            best_model = model





Training with: {'batch_size': 32, 'dropout': 0.1, 'epochs': 50, 'hidden_dim': 64, 'learning_rate': 0.0003, 'num_layers': 2}
Epoch [10/50], Loss: 0.000110
Early stopping at epoch 19 with loss 0.00010595958883641288
Test MSE: 0.000327, R²: -0.1994, MAE: 0.011673

Training with: {'batch_size': 32, 'dropout': 0.1, 'epochs': 50, 'hidden_dim': 64, 'learning_rate': 0.0003, 'num_layers': 3}
Epoch [10/50], Loss: 0.000233
Epoch [20/50], Loss: 0.000095
Early stopping at epoch 28 with loss 0.00011972255742875859
Test MSE: 0.000283, R²: -0.0384, MAE: 0.010589

Training with: {'batch_size': 32, 'dropout': 0.1, 'epochs': 50, 'hidden_dim': 64, 'learning_rate': 0.0005, 'num_layers': 2}
Epoch [10/50], Loss: 0.001491
Epoch [20/50], Loss: 0.000275
Epoch [30/50], Loss: 0.000233
Epoch [40/50], Loss: 0.000140
Epoch [50/50], Loss: 0.000110
Test MSE: 0.000686, R²: -1.5185, MAE: 0.019447

Training with: {'batch_size': 32, 'dropout': 0.1, 'epochs': 50, 'hidden_dim': 64, 'learning_rate': 0.0005, 'num_layers': 3}

## 7. Final Results

In [35]:
print("\n🏆 Best Model Performance:")
print(f"Best MSE: {best_mse:.6f}")
print(f"Best Params: {best_params}")


🏆 Best Model Performance:
Best MSE: 0.000264
Best Params: {'batch_size': 32, 'dropout': 0.1, 'epochs': 50, 'hidden_dim': 128, 'learning_rate': 0.0005, 'num_layers': 2}


### Single Asset Prediction

In [51]:
predictions = {}
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).cpu().numpy().flatten()
    
    # Store predictions
date_index = merged_data.index[-len(y_pred):]
predictions[asset] = pd.Series(y_pred, index=date_index)

In [52]:
predicted_returns = pd.DataFrame(predictions)
predicted_returns

,Gold ETF
Date,
2018-10-15,0.000942
2018-10-16,0.001510
2018-10-17,0.000816
2018-10-18,0.001325
2018-10-19,0.001534
...,...
2020-12-21,-0.000552
2020-12-23,-0.000739
2020-12-24,-0.000459


In [46]:
# Mean-Variance weight strategy
expected_returns = predicted_returns.mean()
volatility = predicted_returns.std()
weights = expected_returns / volatility
weights /= weights.sum()



In [47]:
# Simulate portfolio
daily_returns = predicted_returns @ weights
portfolio_nav = (1 + daily_returns).cumprod()

# Evaluation functions
def sharpe_ratio(returns, risk_free_rate=0.0):
    return (returns.mean() - risk_free_rate) / returns.std()

def max_drawdown(nav):
    peak = nav.cummax()
    return ((nav - peak) / peak).min()

def annualized_return(returns, freq=252):
    return (1 + returns.mean())**freq - 1

def annualized_volatility(returns, freq=252):
    return returns.std() * np.sqrt(freq)

In [53]:

# Final metrics
print("Portfolio Evaluation")
print(f"Sharpe Ratio: {sharpe_ratio(daily_returns):.4f}")
print(f"Max Drawdown: {max_drawdown(portfolio_nav):.2%}")
print(f"Annualized Return: {annualized_return(daily_returns):.2%}")
print(f"Annualized Volatility: {annualized_volatility(daily_returns):.2%}")
print("\nAsset Weights Used:")
print(weights.round(4))


Portfolio Evaluation
Sharpe Ratio: 0.3180
Max Drawdown: -12.21%
Annualized Return: 7.34%
Annualized Volatility: 1.40%

Asset Weights Used:
US Treasury Bonds    1.0
dtype: float32


### Multi Asset Prediction

In [54]:
predictions = {}
assets = ['S&P 500', 'FTSE 100', 'Gold ETF']
for asset in assets:
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor).cpu().numpy().flatten()
        
        # Store predictions
    date_index = merged_data.index[-len(y_pred):]
    predictions[asset] = pd.Series(y_pred, index=date_index)

In [55]:
predicted_returns = pd.DataFrame(predictions)
predicted_returns

,S&P 500,FTSE 100,Gold ETF
Date,,,
2018-10-15,0.000942,0.000942,0.000942
2018-10-16,0.001510,0.001510,0.001510
2018-10-17,0.000816,0.000816,0.000816
2018-10-18,0.001325,0.001325,0.001325
2018-10-19,0.001534,0.001534,0.001534
...,...,...,...
2020-12-21,-0.000552,-0.000552,-0.000552
2020-12-23,-0.000739,-0.000739,-0.000739
2020-12-24,-0.000459,-0.000459,-0.000459


In [56]:
# Mean-Variance weight strategy
expected_returns = predicted_returns.mean()
volatility = predicted_returns.std()
weights = expected_returns / volatility
weights /= weights.sum()
daily_returns = predicted_returns @ weights
portfolio_nav = (1 + daily_returns).cumprod()



In [57]:
# Final metrics
print("Portfolio Evaluation")
print(f"Sharpe Ratio: {sharpe_ratio(daily_returns):.4f}")
print(f"Max Drawdown: {max_drawdown(portfolio_nav):.2%}")
print(f"Annualized Return: {annualized_return(daily_returns):.2%}")
print(f"Annualized Volatility: {annualized_volatility(daily_returns):.2%}")
print("\nAsset Weights Used:")
print(weights.round(4))


Portfolio Evaluation
Sharpe Ratio: 0.3180
Max Drawdown: -12.21%
Annualized Return: 7.34%
Annualized Volatility: 1.40%

Asset Weights Used:
S&P 500     0.3333
FTSE 100    0.3333
Gold ETF    0.3333
dtype: float32
